# Experiment Explorer

In [ ]:
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import importlib
import datetime

sys.path.insert(0, os.path.abspath(r'C:/Users/mmccann/repos/bonhoeffer/prey_capture/'))
import processing_parameters
import functions_bondjango as bd
import functions_plotting as fp
import functions_data_handling as fdh
import functions_kinematic as fk

In [ ]:
class ExperimentViewer():
    def __init__(self, experiment_type, init_query):
        self.query = None
        self.dataset = None

        # Set up GUI
        self.available_experiments = {}
        self._get_query_results(experiment_type, init_query)

        self.miceW = widgets.Select(options=self.available_experiments.keys())
        init_mouse = self.miceW.value
        self.dateW = widgets.Select(options=self.available_experiments[init_mouse].keys())
        init_date = self.dateW.value
        self.rigW = widgets.Select(options=self.available_experiments[init_mouse][init_date])

        k = widgets.interactive(self.generate_query, mouse=self.miceW, day=self.dateW, rig=self.rigW)
        j = widgets.interactive(self._select_rig, mouse=self.miceW, day=self.dateW)
        i = widgets.interactive(self._select_date, mouse=self.miceW)
        
        display(k)

    def generate_query(self, mouse, day, rig):
        day_format = datetime.datetime.strptime(day, '%Y-%m-%d').strftime('%m_%d_%Y')
        self.query = f'mouse:{mouse}, rig:{rig}, slug:{day_format}'
        return self.query
        
    def _get_query_results(self, experiment_type, query):
        entries = bd.query_database(experiment_type, query)
        mice = np.unique([entry['mouse'] for entry in entries])

        available_dates = {}
        for mouse in mice:
            short_dates = []
            for entry in entries:
                if entry['mouse'] == mouse:
                    short_dates.append(entry['date'][:10])
            available_dates[mouse] = np.unique(short_dates)

        for mouse, dates in available_dates.items():
            self.available_experiments[mouse] = {}
            for date in dates:
                rigs = []
                for entry in entries:
                    if (entry['mouse'] == mouse) and (date in entry['date']):
                        rigs.append(entry['rig'])

                if len(rigs) > 0:
                    self.available_experiments[mouse].update({date: rigs})

    def _select_date(self, mouse):
        self.dateW.options =self.available_experiments[mouse].keys()

    def _select_rig(self, mouse, day):
        self.rigW.options = self.available_experiments[mouse][day]
    
        

In [ ]:
f = ExperimentViewer('vr_experiment', 'imaging:wirefree')

In [ ]:
importlib.reload(processing_parameters)

# get the search string
search_string = processing_parameters.search_string + r", analysis_type:preprocessing"

# get the paths from the database
file_path, paths_all, parsed_query, date_list, animal_list = fdh.fetch_preprocessing(search_string)

animal_idxs = [i for i,d in enumerate(animal_list) if d==parsed_query['mouse'].lower()]
good_entries = [file_path[index] for index in animal_idxs]
input_path = [paths_all[index] for index in animal_idxs]

# # assemble the output path
# out_path = os.path.join(paths.analysis_path, 'test_latentconsolidate.hdf5')
print(input_path)